In [68]:
import os
import re
import csv
from PIL import Image

In [69]:
import pandas as pd
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np

In [70]:
im_path = "/home/bmazoyer/Documents/Data/police/medias_unique/"

In [71]:
n_sample = 500

In [72]:
def get_vector(input_image):
    image = input_image.convert("RGB")  # in case input image is not in RGB format
    img_t = transform(image)
    batch_t = torch.unsqueeze(img_t, 0)
    my_embedding = torch.zeros([1, 512, 1, 1])
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    h = layer.register_forward_hook(copy_data)
    model(batch_t)
    h.remove()
    return my_embedding.squeeze().cpu().numpy()

In [73]:
model = models.resnet18(pretrained=True)
layer = model._modules.get('avgpool')

In [74]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])

In [75]:
im_names = [os.path.join(root, name)
            for root, dirs, files in os.walk(im_path)
            for name in files]

In [76]:
existing_images_df = pd.DataFrame([re.findall(r"[\w']+", im_name)[1:3] for im_name in im_names],
                                  columns=['cat_id', 'pid'])
existing_images_df['impath'] = im_names
existing_images_df = existing_images_df.sample(n_sample)

In [77]:
vecs = [list(get_vector(Image.open(impath))) for _, pid, impath in existing_images_df.values]

In [78]:
with open('vis/feature_vecs.tsv', 'w') as fw:
    csv_writer = csv.writer(fw, delimiter='\t')
    csv_writer.writerows(vecs)

In [79]:
images = [Image.open(filename).resize((300,300)) for filename in existing_images_df['impath']]

In [80]:
image_width, image_height = images[0].size

In [81]:
one_square_size = int(np.ceil(np.sqrt(len(images))))
master_width = (image_width * one_square_size) 
master_height = image_height * one_square_size

In [82]:
spriteimage = Image.new(
    mode='RGBA',
    size=(master_width, master_height),
    color=(0,0,0,0))  # fully transparent

In [83]:
for count, image in enumerate(images):
    div, mod = divmod(count,one_square_size)
    h_loc = image_width*div
    w_loc = image_width*mod    
    spriteimage.paste(image,(w_loc,h_loc))

In [84]:
spriteimage.convert("RGB").save('vis/sprite.jpg', transparency=0)

In [47]:
embeddings {
  tensor_path: "feature_vecs.tsv"
  sprite {
    image_path: "sprite.jpg"
    single_image_dim: 50
    single_image_dim: 50
  }
}

SyntaxError: invalid syntax (<ipython-input-47-bfd0f17e5484>, line 1)